In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Setup**

In [ ]:
pip install -U tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.3/612.3 kB 10.3 MB/s eta 0:00:00


In [ ]:
import os, random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import InputLayer, Conv1D, SeparableConv1D, Dense, Flatten, MaxPooling1D, LSTM
from keras.models import Sequential
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from keras.callbacks import CSVLogger
from matplotlib.colors import ListedColormap
from IPython.display import Image, display
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score
from mlxtend.plotting import plot_confusion_matrix
from sklearn.manifold import TSNE
%matplotlib inline

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


**Prepare the data**

In [ ]:
# x_train = np.load('/content/drive/MyDrive/Datasets/Hydrophobicity/x_train.npy')
# y_train = np.load('/content/drive/MyDrive/Datasets/Hydrophobicity/y_train.npy')
# x_test = np.load('/content/drive/MyDrive/Datasets/Hydrophobicity/x_test.npy')
# y_test = np.load('/content/drive/MyDrive/Datasets/Hydrophobicity/y_test.npy')

import pickle
X_Temp = open('/content/drive/MyDrive/Datasets/Osteosarcoma_Dataset/x_data','rb')
x_data_np = pickle.load(X_Temp)

Y_Temp = open('/content/drive/MyDrive/Datasets/Osteosarcoma_Dataset/y_data','rb')
y_data_np = pickle.load(Y_Temp)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data_np, y_data_np, test_size=0.2,random_state=101)

In [ ]:
num_classes = 3# classes of data
y_train = keras.utils.to_categorical(y_train, num_classes)
y_train = np.where(y_train==1)[1]
y_test = keras.utils.to_categorical(y_test, num_classes)
y_test = np.where(y_test==1)[1]

input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
num_classes = len(np.unique(y_train))

print('Test Shape', x_test.shape, y_test.shape)
print('Train Shape',x_train.shape, y_train.shape)

Test Shape (219, 128, 128, 3) (219,)
Train Shape (872, 128, 128, 3) (872,)


In [ ]:
from collections import Counter
c = Counter(y_train)
print(c.items())

dict_items([(1, 218), (0, 413), (2, 241)])


**Configure the hyperparameters**

In [ ]:
input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])
num_classes = len(np.unique(y_train))
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128  # 256
num_epochs = 500  # 500
image_size = 64  # We'll resize input images to this size   # 64
patch_size = 6  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 12
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier

**Use data augmentation**

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)

**Implement multilayer perceptron (MLP)**

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

**Implement patch creation as a layer**

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

**Implement the patch encoding layer:** The PatchEncoder layer will linearly transform a patch by projecting it into a vector of size projection_dim. In addition, it adds a learnable position embedding to the projected vector.

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

**Build the ViT model**

In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(augmented)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

# ___________________________ contribution ___________________________
    x = layers.SeparableConv1D(projection_dim,3,1,dilation_rate=1)(encoded_patches)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Dropout(0.1)(x)
    encoded_patches = x
# ___________________________ contribution ___________________________

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

## Compile, train, and evaluate the mode

In [ ]:
def run_experiment(model):
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(1, name="top-1-accuracy"),
        ],
    )

    checkpoint_filepath = "/content/drive/MyDrive/Datasets/Hydrophobicity/Model2.h5"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    # history = model.fit(
    #     x=x_train,
    #     y=y_train,
    #     batch_size=batch_size,
    #     epochs=num_epochs,
    #     validation_split=0.15,
    #     shuffle=True,
    #     steps_per_epoch=19,
    #     callbacks=[checkpoint_callback],
    # )

    history = model.fit(
        x=x_train,
        y=y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=(x_test, y_test),
        shuffle=True,
        callbacks=[checkpoint_callback],
    )

    model.load_weights(checkpoint_filepath)
    _, accuracy, top_5_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history

vit_classifier = create_vit_classifier()

In [ ]:
# model.summary()

In [ ]:
history = run_experiment(vit_classifier)

Epoch 1/500
7/7 [==============================] - 20s 588ms/step - loss: 5.5927 - accuracy: 0.3521 - top-1-accuracy: 0.3521 - val_loss: 1.7945 - val_accuracy: 0.5799 - val_top-1-accuracy: 0.5799
Epoch 2/500
7/7 [==============================] - 1s 207ms/step - loss: 1.6246 - accuracy: 0.4679 - top-1-accuracy: 0.4679 - val_loss: 1.1023 - val_accuracy: 0.4840 - val_top-1-accuracy: 0.4840
Epoch 3/500
7/7 [==============================] - 2s 267ms/step - loss: 1.1530 - accuracy: 0.5183 - top-1-accuracy: 0.5183 - val_loss: 0.8633 - val_accuracy: 0.6347 - val_top-1-accuracy: 0.6347
Epoch 4/500
7/7 [==============================] - 2s 304ms/step - loss: 0.8964 - accuracy: 0.6250 - top-1-accuracy: 0.6250 - val_loss: 0.6594 - val_accuracy: 0.7078 - val_top-1-accuracy: 0.7078
Epoch 5/500
7/7 [==============================] - 2s 231ms/step - loss: 0.7302 - accuracy: 0.7202 - top-1-accuracy: 0.7202 - val_loss: 0.7548 - val_accuracy: 0.6895 - val_top-1-accuracy: 0.6895
Epoch 6/500
7/7 [=======

In [ ]:
# model = vit_classifier

In [ ]:
# model.summary()

In [ ]:
#y_pred = model.predict([x_test, y_test])
print(history.history.keys())
with open('/content/drive/MyDrive/Datasets/Osteosarcoma_Dataset/val_loss.txt', 'wb') as a:
    np.savetxt(a, history.history['val_loss'])
with open('/content/drive/MyDrive/Datasets/Osteosarcoma_Dataset/train_loss.txt', 'wb') as b:
    np.savetxt(b, history.history['loss'])
with open('/content/drive/MyDrive/Datasets/Osteosarcoma_Dataset/train_accuracy.txt', 'wb') as c:
    np.savetxt(c, history.history['accuracy'])
with open('/content/drive/MyDrive/Datasets/Osteosarcoma_Dataset/val_accuracy.txt', 'wb') as d:
    np.savetxt(d, history.history['val_accuracy'])
with open('/content/drive/MyDrive/Datasets/Osteosarcoma_Dataset/train_top-1-accuracy.txt', 'wb') as e:
    np.savetxt(e, history.history['top-1-accuracy'])
with open('/content/drive/MyDrive/Datasets/Osteosarcoma_Dataset/val_top_1_accuracy.txt', 'wb') as f:
    np.savetxt(f, history.history['val_top-1-accuracy'])

dict_keys(['loss', 'accuracy', 'top-1-accuracy', 'val_loss', 'val_accuracy', 'val_top-1-accuracy'])


**Classification Performances**

In [ ]:
# y_pred = vit_classifier.predict(x_test)
# rounded_prediction = np.argmax(y_pred, axis=-1)
# cm = confusion_matrix(y_true=y_test, y_pred=rounded_prediction)

# fig, ax = plot_confusion_matrix(conf_mat=cm ,  figsize=(5, 5))
# plt.show()

#print(label_predicted)
# cm_plot_labels = ['1', '2', '3']
label_predicted = vit_classifier.predict(x_test)
rounded_prediction = np.argmax(label_predicted, axis=-1)
cm = confusion_matrix(y_true=y_test, y_pred=rounded_prediction)
cm
from sklearn.metrics import classification_report
# print('\nClassification Report\n')
# print(classification_report(y_test, rounded_prediction, target_names=cm_plot_labels))
with open('/content/drive/MyDrive/Datasets/Osteosarcoma_Dataset/y_test_rounded.txt', 'wb') as i:
    np.savetxt(i,y_test)
with open('/content/drive/MyDrive/Datasets/Osteosarcoma_Dataset/Rounded_prediction.txt', 'wb') as j:
    np.savetxt(j,rounded_prediction)

7/7 [==============================] - 3s 43ms/step


In [ ]:
# from keras.utils.vis_utils import plot_model
# plot_model(model, to_file='/content/drive/MyDrive/Colab Notebooks/Hydrophobicity/ViT_model_plot.png', show_shapes=True, show_layer_names=True)